In [1]:
import copy, math
import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(suppress=True)

In [5]:
# function to load the data
def load_house_data():
    data = np.loadtxt("./Week2/data/houses.txt", delimiter=',', skiprows=1)
    X = data[:, :4]
    y = data[:, 4]
    return X, y

In [18]:
# X_train, y_train = load_house_data()
X_train = np.array([[2104, 5, 1, 45], [1416, 3, 2, 40], [852, 2, 1, 35]])
y_train = np.array([460, 232, 178])

In [3]:
# for demo'ing w and b will be loaded with initial values that are near the optimal. w is a 1-d NumPy vector
b_init = 785.1811367994083
w_init = np.array([0.39133535, 18.75376741, -53.36032453, -26.42131618])
print(f"w_init shape: {w_init.shape}, b_init type: {type(b_init)}")

w_init shape: (4,), b_init type: <class 'float'>


The model's prediction with multiple variables is given by the linear model:

$$ f_{\mathbf{w},b}(\mathbf{x}) =  w_0x_0 + w_1x_1 +... + w_{n-1}x_{n-1} + b \tag{1}$$
or in vector notation:
$$ f_{\mathbf{w},b}(\mathbf{x}) = \mathbf{w} \cdot \mathbf{x} + b  \tag{2} $$ 
where $\cdot$ is a vector `dot product`

In [4]:
# equation (1), no dot notation
def predict_single_loop(x, w, b):
    """
    single predict using linear regression

    Args:
        x (ndarray): Shape (n,) example with multiple features
        w (ndarray): Shape (n,) model parameters
        b (scalar): model parameter

    Reurns:
        p (scalar): prediction
    """
    n = x.shape[0]
    p = 0.0
    for i in range(n):
        p_i = x[i] * w[i]
        p = p + p_i
    p = p + b
    return p

In [5]:
# equation 1(), with dot notation
def predict(x, w, b):
    """
    single predict using linear regression
    Args:
        x (ndarray): Shape (n,) example with multiple features
        w (ndarray): Shape (n,) model parameters
        b (scalar): model parameter

    Reurns:
        p (scalar): prediction
    """
    p = np.dot(x, w) + b
    return p

In [6]:
# make a prediction, using 1 row from the training data
x_vec = X_train[0,:]
print(f"x_vec shape {x_vec.shape}, x_vec value {x_vec}")

# make a prediction
f_wb = predict(x_vec, w_init, b_init)
print(f"f_wb shape: {f_wb.shape}, prediction {f_wb}")

x_vec shape (4,), x_vec value [2104    5    1   45]
f_wb shape: (), prediction 459.9999976194082


# 4 Compute Cost With Multiple Variables
The equation for the cost function with multiple variables $J(\mathbf{w},b)$ is:
$$J(\mathbf{w},b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})^2 \tag{3}$$ 
where:
$$ f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)} + b  \tag{4} $$ 

In [7]:
def compute_cost(X, y, w, b):
    """
    compute cost
    Args:
        X (ndarray (m, n)): Data, m examples with n features
        y (ndarray (m,)): target values
        w (ndarray (n,)): model parameters
        b (scalar): model parameter

    Returns:
        cost (scalar): cost
    """
    m = X.shape[0]
    cost = 0.0
    for i in range(m):
        f_wb_i = np.dot(X[i], w) + b # (4) above
        cost = cost + (f_wb_i - y[i])**2 # summation in (3) above
    cost = cost / (2 * m) # division in (3) above
    return cost

In [20]:
cost = compute_cost(X_train, y_train, w_init, b_init)
print(f"Cost at optimal w: {cost}")

Cost at optimal w: 1.5578904330213735e-12


# Derivates for gradient descent
$$
\begin{align}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \tag{6}  \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{7}
\end{align}
$$

## 5.1 Compute Gradient with Multiple Variables
An implementation for calculating the equations (6) and (7) is below. There are many ways to implement this. In this version, there is an
- outer loop over all m examples. 
    - $\frac{\partial J(\mathbf{w},b)}{\partial b}$ for the example can be computed directly and accumulated
    - in a second loop over all n features:
        - $\frac{\partial J(\mathbf{w},b)}{\partial w_j}$ is computed for each $w_j$.

In [21]:
def compute_gradient(X, y, w, b):
    """
    Computes the gradient for linear regression
    Args:
        X (ndarray (m,n)): Data, m examples with n features
        y (ndarray (n,)): target values
        w (ndarray (n,)): model parameters
        b (scalar): model parameter
    
    Returns:
        dj_dw (ndarray (n,)): The gradient of the cost w.r.t. parameters w
        dj_db (scalar): The gradient of the cost w.r.t. parameter b
    """
    m, n = X.shape # number of examples, number of features
    dj_dw = np.zeros((n,))
    dj_db = 0.0

    for i in range(m):
        err = (np.dot(X[i], w) + b) - y[i] # equation (7) above
        for j in range(n):
            dj_dw[j] = dj_dw[j] + err * X[i, j] # equation (6)
        dj_db = dj_db + err
    dj_dw = dj_dw / m
    dj_db = dj_db / m

    return dj_db, dj_dw

In [22]:
# compute and display the gradient
tmp_dj_db, tmp_dj_dw = compute_gradient(X_train, y_train, w_init, b_init)
print(f"dj_db at initial w, b: {tmp_dj_db}")
print(f"dj_dw at initial w, b: {tmp_dj_dw}")

dj_db at initial w, b: -1.6739251122999121e-06
dj_dw at initial w, b: [-0.00272624 -0.00000627 -0.00000222 -0.00006924]


## Gradient Descent With Multiple Variables

Gradient descent for multiple variables:

$$\begin{align*} \text{repeat}&\text{ until convergence:} \; \lbrace \newline\;
& w_j = w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{5}  \; & \text{for j = 0..n-1}\newline
&b\ \ = b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b}  \newline \rbrace
\end{align*}$$


In [13]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters):
    """
    Performs batch gradient descent to learn w and b. Updates w and b by taking
    num_iuters gradient steps with learning rate alpha.

    Args:
        X (ndarray (m,n))   : Data, m examples with n features
        y (ndarray (m,))    : Target values
        w_in (ndarray (n,)) : initial model parameters
        b_in (scalar)       : initial model parameter
        cost_function       : function to compute cost
        gradient_function   : function to compute the gradient
        alpha (float)       : learning rate
        num_iters (int)     : number of iterations

    Returns:
        w (ndarray (n,)) : Updated values of the parameters
        b (scalar)       : Updated value of the parameter
    """
    # an array to store cost J and w's at each iteration for graphing later
    J_history = []
    w = copy.deepcopy(w_in) # deepcopy to avoid modifying global w
    b = b_in

    for i in range(num_iters):
        # calculate the gradient and update the parameters
        dj_db, dj_dw = gradient_function(X, y, w, b)

        # update parameters using w, b, alpha and grdient
        w =  w - alpha * dj_dw
        b = b - alpha * dj_db

        # save the cost J at each iteration
        if i < 100000: # to prevent resource exhaustion
            J_history.append(cost_function(X, y, w, b))

        # print cost every at 10 intervals 10 times or as many iterations if < 10
        if i % math.ceil(num_iters / 10) == 0:
            print(f"Iteration {i:4d}: Cost {J_history[-1]:8.2f}   ")

    return w, b, J_history # J_history for graphing


In [23]:
# initialize parameters
initial_w = np.zeros_like(w_init)
initial_b = 0.
# some gradient descent settings
iterations = 1000
alpha = 0.00000050
# run gradient descent
w_final, b_final, J_hist = gradient_descent(X_train, y_train, initial_w, initial_b,
                                            compute_cost, compute_gradient,
                                            alpha, iterations)
print(f"b, w found by gradient descent: {b_final:0.2f}, {w_final} ")
m, _ = X_train.shape
for i in range(m):
    print(f"prediction: {np.dot(X_train[i], w_final) + b_final:0.2f}, target value: {y_train[i]}")

Iteration    0: Cost  2529.46   
Iteration  100: Cost   695.99   
Iteration  200: Cost   694.92   
Iteration  300: Cost   693.86   
Iteration  400: Cost   692.81   
Iteration  500: Cost   691.77   
Iteration  600: Cost   690.73   
Iteration  700: Cost   689.71   
Iteration  800: Cost   688.70   
Iteration  900: Cost   687.69   
b, w found by gradient descent: -0.00, [ 0.20396569  0.00374919 -0.0112487  -0.0658614 ] 
prediction: 426.19, target value: 460
prediction: 286.17, target value: 232
prediction: 171.47, target value: 178
